In [0]:
storage_account_name = "adls1046project"
container_name = "container"
file_name = "GlobalLandTemperaturesByCountry.csv"
storage_account_access_key = "KUKAhIfCYt/M/1Co79hLgWIsSzICBxQ7TJQ3s7DwtrO1gZWe/lRA0Rh43+Qmu1+PR31WfvqtRx6w+ASthC5Ofg=="
spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)
data = spark.read.format("csv").load(f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{file_name}", inferSchema=True, header=True)
# Register the DataFrame as a SQL temporary view
data.createOrReplaceTempView("climate_data")

In [0]:
# Handle missing values
# Drop rows with any missing temperature values
cleaned_data = data.dropna(subset=["AverageTemperature"])  
cleaned_data.show()

+----------+-------------------+-----------------------------+-------+
|      Date| AverageTemperature|AverageTemperatureUncertainty|Country|
+----------+-------------------+-----------------------------+-------+
|1743-11-01| 4.3839999999999995|                        2.294|  Åland|
|1744-04-01|               1.53|                         4.68|  Åland|
|1744-05-01|  6.702000000000001|                        1.789|  Åland|
|1744-06-01| 11.609000000000002|                        1.577|  Åland|
|1744-07-01|             15.342|                         1.41|  Åland|
|1744-09-01|             11.702|                        1.517|  Åland|
|1744-10-01|              5.477|                        1.862|  Åland|
|1744-11-01|              3.407|                        1.425|  Åland|
|1744-12-01|             -2.181|                        1.641|  Åland|
|1745-01-01|              -3.85|                        1.841|  Åland|
|1745-02-01| -6.574999999999998|                         1.36|  Åland|
|1745-

In [0]:
# Drop rows with any missing date values
cleaned_data = cleaned_data.dropna(subset=["Date"])
cleaned_data.show()

+----------+------------------+-----------------------------+-------+
|      Date|AverageTemperature|AverageTemperatureUncertainty|Country|
+----------+------------------+-----------------------------+-------+
|1743-11-01|             4.384|                        2.294|  Åland|
|1744-04-01|              1.53|                         4.68|  Åland|
|1744-05-01|             6.702|                        1.789|  Åland|
|1744-06-01|            11.609|                        1.577|  Åland|
|1744-07-01|            15.342|                         1.41|  Åland|
|1744-09-01|            11.702|                        1.517|  Åland|
|1744-10-01|             5.477|                        1.862|  Åland|
|1744-11-01|             3.407|                        1.425|  Åland|
|1744-12-01|            -2.181|                        1.641|  Åland|
|1745-01-01|             -3.85|                        1.841|  Åland|
|1745-02-01|            -6.575|                         1.36|  Åland|
|1745-03-01|        

In [0]:
# Remove rows with missing country values
from pyspark.sql.functions import col
cleaned_data = cleaned_data.filter(col("Country").isNotNull())  
cleaned_data.show()

+----------+-------------------+-----------------------------+-------+
|      Date| AverageTemperature|AverageTemperatureUncertainty|Country|
+----------+-------------------+-----------------------------+-------+
|1743-11-01| 4.3839999999999995|                        2.294|  Åland|
|1744-04-01|               1.53|                         4.68|  Åland|
|1744-05-01|  6.702000000000001|                        1.789|  Åland|
|1744-06-01| 11.609000000000002|                        1.577|  Åland|
|1744-07-01|             15.342|                         1.41|  Åland|
|1744-09-01|             11.702|                        1.517|  Åland|
|1744-10-01|              5.477|                        1.862|  Åland|
|1744-11-01|              3.407|                        1.425|  Åland|
|1744-12-01|             -2.181|                        1.641|  Åland|
|1745-01-01|              -3.85|                        1.841|  Åland|
|1745-02-01| -6.574999999999998|                         1.36|  Åland|
|1745-

In [0]:
# Drop irrelevant columns
cleaned_data = cleaned_data.drop("AverageTemperatureUncertainty")
cleaned_data.show()

+----------+------------------+-------+
|      Date|AverageTemperature|Country|
+----------+------------------+-------+
|1743-11-01|             4.384|  Åland|
|1744-04-01|              1.53|  Åland|
|1744-05-01|             6.702|  Åland|
|1744-06-01|            11.609|  Åland|
|1744-07-01|            15.342|  Åland|
|1744-09-01|            11.702|  Åland|
|1744-10-01|             5.477|  Åland|
|1744-11-01|             3.407|  Åland|
|1744-12-01|            -2.181|  Åland|
|1745-01-01|             -3.85|  Åland|
|1745-02-01|            -6.575|  Åland|
|1745-03-01|            -4.195|  Åland|
|1745-04-01|            -0.966|  Åland|
|1750-01-01|             1.091|  Åland|
|1750-02-01|             0.809|  Åland|
|1750-03-01|             0.923|  Åland|
|1750-04-01|             3.943|  Åland|
|1750-05-01|             6.265|  Åland|
|1750-06-01|            12.408|  Åland|
|1750-07-01|            16.683|  Åland|
+----------+------------------+-------+
only showing top 20 rows



In [0]:
# Remove Duplicates
cleaned_data = cleaned_data.dropDuplicates()
cleaned_data.show()

+----------+------------------+-------+
|      Date|AverageTemperature|Country|
+----------+------------------+-------+
|1745-03-01|            -4.195|  Åland|
|1750-02-01|             0.809|  Åland|
|1743-11-01|             4.384|  Åland|
|1750-08-01|            17.076|  Åland|
|1750-05-01|             6.265|  Åland|
|1744-12-01|            -2.181|  Åland|
|1750-03-01|             0.923|  Åland|
|1744-05-01|             6.702|  Åland|
|1745-04-01|            -0.966|  Åland|
|1744-10-01|             5.477|  Åland|
|1750-07-01|            16.683|  Åland|
|1744-11-01|             3.407|  Åland|
|1750-01-01|             1.091|  Åland|
|1744-04-01|              1.53|  Åland|
|1744-06-01|            11.609|  Åland|
|1750-06-01|            12.408|  Åland|
|1744-07-01|            15.342|  Åland|
|1745-02-01|            -6.575|  Åland|
|1744-09-01|            11.702|  Åland|
|1745-01-01|             -3.85|  Åland|
+----------+------------------+-------+
only showing top 20 rows



In [0]:
# Handle inconsistencies
# Convert decimals to only 2 digits
cleaned_data = cleaned_data.withColumn("AverageTemperature", round((cleaned_data["AverageTemperature"]),2))
cleaned_data.show()

+----------+------------------+-------+
|      Date|AverageTemperature|Country|
+----------+------------------+-------+
|1745-03-01|              -4.2|  Åland|
|1750-02-01|              0.81|  Åland|
|1743-11-01|              4.38|  Åland|
|1750-08-01|             17.08|  Åland|
|1750-05-01|              6.27|  Åland|
|1744-12-01|             -2.18|  Åland|
|1750-03-01|              0.92|  Åland|
|1744-05-01|               6.7|  Åland|
|1745-04-01|             -0.97|  Åland|
|1744-10-01|              5.48|  Åland|
|1750-07-01|             16.68|  Åland|
|1744-11-01|              3.41|  Åland|
|1750-01-01|              1.09|  Åland|
|1744-04-01|              1.53|  Åland|
|1744-06-01|             11.61|  Åland|
|1750-06-01|             12.41|  Åland|
|1744-07-01|             15.34|  Åland|
|1745-02-01|             -6.58|  Åland|
|1744-09-01|              11.7|  Åland|
|1745-01-01|             -3.85|  Åland|
+----------+------------------+-------+
only showing top 20 rows

